In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
def create_pop_csv(filename="data/processed_pop.csv"):
    """
    creates a csv compiling population and mesh block data
    outputs to data/out.csv
    """

    # first compile mesh block populations
    dfs = pd.read_excel(
        "data/Mesh Block Counts, 2021.xlsx", 
        sheet_name=None, 
        skiprows=6) # skiprows to skip the headers etc in the spreadsheet
    
    dfs.pop("Contents") # get rid of contents page from xlsx
    pop_counts = pd.concat(dfs.values())[["MB_CODE_2021", "Person"]] # combine all pages, keep relevant cols

    # remove non-numeric values for MB code. this is text in the data see the xlsx
    pop_counts["MB_CODE_2021"] = pop_counts["MB_CODE_2021"].astype(str)
    pop_counts = pop_counts[pop_counts["MB_CODE_2021"].str.isnumeric()]
    
    # -------------------------------------------------------------------------------------------------
    # now we have a clean dataframe of populations in each mesh block
    # let's make a clean dataframe of mesh block locations (centroids), then combine
    mesh_blocks = gpd.read_file("data/MB_2021_AUST_GDA2020.shp")
    mesh_blocks["MB_CODE_2021"] = mesh_blocks["MB_CODE21"] # rename col to be the same as other df

    # remove non-numeric values as with other df
    mesh_blocks["MB_CODE_2021"] = mesh_blocks["MB_CODE_2021"].astype(str)
    mesh_blocks = mesh_blocks[mesh_blocks["MB_CODE_2021"].str.isnumeric()]

    # compute centroids
    centroids = mesh_blocks.geometry.centroid # centroid is not technically correct but curvature over small size is small

    # extract lat / lon
    mesh_blocks["lat"] = centroids.y
    mesh_blocks["lon"] = centroids.x
    mesh_blocks = mesh_blocks[["MB_CODE_2021", "lat", "lon"]]
    # make sure we don't have any values in either table that is not in the other
    only_df1 = set(pop_counts["MB_CODE_2021"]) - set(mesh_blocks["MB_CODE_2021"])
    only_df2 = set(mesh_blocks["MB_CODE_2021"]) - set(pop_counts["MB_CODE_2021"])
    if (only_df1 or only_df2):
        raise ValueError("Your data sucks and is inconsistent")
    pop_counts = pop_counts.merge(mesh_blocks, on="MB_CODE_2021", how="left")
    pop_counts.to_csv(filename,index=False)

In [3]:
create_pop_csv()

C:\Users\2468t\AppData\Local\Temp\ipykernel_11336\2328376001.py:31: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = mesh_blocks.geometry.centroid # centroid is not technically correct but curvature over small size is small
